In [1]:
import pandas as pd
import polars as pl
import zipfile
import os
import pypyodbc as odbc
from dotenv import load_dotenv
from sqlalchemy import create_engine
import urllib

In [2]:
load_dotenv()

True

In [3]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = 'DESKTOP-7IM04QA'
DATABASE_NAME = 'ReceitaFederal'

connection_string = f"""
DRIVER={{{DRIVER_NAME}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
Trusted_Connection=yes;
"""

conn = odbc.connect(connectString=connection_string)
print(conn)

In [4]:
DRIVER = "ODBC Driver 17 for SQL Server"
SERVER = "DESKTOP-7IM04QA"
DATABASE = "ReceitaFederal"
USERNAME = os.environ.get("user")
PASSWORD = os.environ.get("password")

params = urllib.parse.quote_plus(
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [5]:
columnsEstabelecimentos = [
    "CNPJ BÁSICO",
    "CNPJ ORDEM",
    "CNPJ DV",
    "IDENTIFICADOR MATRIZ/FILIAL",
    "NOME FANTASIA",
    "SITUAÇÃO CADASTRAL",
    "DATA SITUAÇÃO CADASTRAL",
    "MOTIVO SITUAÇÃO CADASTRAL",
    "NOME DA CIDADE NO EXTERIOR",
    "PAIS",
    "DATA DE INÍCIO ATIVIDADE",
    "CNAE FISCAL PRINCIPAL",
    "CNAE FISCAL SECUNDÁRIA",
    "TIPO DE LOGRADOURO",
    "LOGRADOURO",
    "NÚMERO",
    "COMPLEMENTO",
    "BAIRRO",
    "CEP",
    "UF",
    "MUNICÍPIO",
    "DDD 1",
    "TELEFONE 1",
    "DDD 2",
    "TELEFONE 2",
    "DDD DO FAX",
    "FAX",
    "CORREIO ELETRÔNICO",
    "SITUAÇÃO ESPECIAL",
    "DATA DA SITUAÇÃO ESPECIAL"
]

columnsEmpresas = [
    "CNPJ BÁSICO",
    "RAZÃO SOCIAL / NOME EMPRESARIAL",
    "NATUREZA JURÍDICA",
    "QUALIFICAÇÃO DO RESPONSÁVEL",
    "CAPITAL SOCIAL DA EMPRESA",
    "PORTE DA EMPRESA",
    "ENTE FEDERATIVO RESPONSÁVEL"
]

columnsSimples = [
    "CNPJ BÁSICO",
    "OPÇÃO PELO SIMPLES",
    "DATA DE OPÇÃO PELO SIMPLES",
    "DATA DE EXCLUSÃO DO SIMPLES",
    "OPÇÃO PELO MEI",
    "DATA DE OPÇÃO PELO MEI",
    "DATA DE EXCLUSÃO DO MEI"
]

In [6]:
pathEmpresas = "DadosRF/Empresas"
pathEstabelecimentos = "DadosRF/Estabelecimentos"
pathSimples = "DadosRF/Simples"

zipfilesEmpresas = os.listdir(pathEmpresas)
zipFilesEstabelecimentos = os.listdir(pathEstabelecimentos)
zipFilesSimples = os.listdir(pathSimples)

In [7]:
def FixPath(files, basePath):
    filesPath = []
    for file in files:
        filesPath.append(os.path.join(basePath, file)) 
        
    return filesPath 

In [8]:
filesEmpresas = FixPath(zipfilesEmpresas, pathEmpresas)
fileEstabelecimentos = FixPath(zipFilesEstabelecimentos, pathEstabelecimentos)
fileSimples = FixPath(zipFilesSimples, pathSimples)

In [51]:
for file in filesEmpresas:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/EmpresasUnzip")

for file in fileEstabelecimentos:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/EstabelecimentosUnzip")

for file in fileSimples:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/SimplesUnzip")


In [9]:
filesEmpresasUnzip = os.listdir("TratamentoInicial/EmpresasUnzip")
fileEstabelecimentosUnzip = os.listdir("TratamentoInicial/EstabelecimentosUnzip")
fileSimplesUnzip = os.listdir("TratamentoInicial/SimplesUnzip")

filesEmpresas = FixPath(filesEmpresasUnzip, "TratamentoInicial/EmpresasUnzip")
fileEstabelecimentos = FixPath(fileEstabelecimentosUnzip, "TratamentoInicial/EstabelecimentosUnzip")
fileSimples = FixPath(fileSimplesUnzip, "TratamentoInicial/SimplesUnzip")


In [10]:
def infer_sql_type(series: pd.Series) -> str:
    
    """
    Infere o tipo SQL Server a partir de uma Series do pandas.
    """
    if pd.api.types.is_integer_dtype(series):
        return "BIGINT"
    elif pd.api.types.is_float_dtype(series):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(series):
        return "BIGINT"
    else:
        # Para strings, podemos usar NVARCHAR(600) por padrão
        max_len = series.dropna().astype(str).map(len).max()
        length = max(600, max_len)  # mínimo 600
        return f"NVARCHAR({length})"

In [ ]:
def bulk_insert(data_file, table, column_names):
    table = table + "V2"
    print("a")
    with open(data_file, "r", encoding="latin1") as f:
        total_lines = sum(1 for _ in f)
    print("b")

    df_sample = pd.read_csv(
        data_file,
        sep=';',
        names=column_names,
        encoding='latin1',
        skiprows=range(1, total_lines-100),  # pula todas menos cabeçalho + últimas 100
        nrows=100
    )
    
    columns_def = ", ".join(f"[{col}] {infer_sql_type(df_sample[col])}" for col in column_names)
    
    print(columns_def)

    sql = f"""
    IF OBJECT_ID('{table}', 'U') IS NULL
    BEGIN
        -- Cria a tabela e insere os dados de uma vez
        SELECT *
        INTO {table}
        FROM OPENROWSET(
            BULK '{data_file}',
            FORMAT = 'CSV',
            FIRSTROW = 2,
            FIELDTERMINATOR = ';',
            ROWTERMINATOR = '0x0a'
        ) AS DataFile;
    END
    ELSE
    BEGIN
        -- Apenas insere (append) na tabela já existente
        INSERT INTO {table}
        SELECT *
        FROM OPENROWSET(
            BULK '{data_file}',
            FORMAT = 'CSV',
            FIRSTROW = 2,
            FIELDTERMINATOR = ';',
            ROWTERMINATOR = '0x0a'
        ) AS DataFile;
    END;
    """.strip()
    
    return sql

In [12]:
def insertFiles(data_files, table, connection, column_names):

    cursor = connection.cursor()
    try:
        print("Begin insert operation")
        with cursor:
            print("Cursor avaliable")
            for data_file in data_files:
                print("checking", os.path.abspath(data_file))
                print("Inserting", os.path.abspath(data_file))
                cursor.execute(bulk_insert(os.path.abspath(data_file), table, column_names))
                print(data_file, "inserted")
            cursor.commit()
    except Exception as err:
        print(err)
        connection.rollback()
        print("Rollback has been made")

In [ ]:
insertFiles(filesEmpresas, "Empresas", conn, columnsEmpresas)

insertFiles(fileEstabelecimentos, "Estabelecimentos", conn, columnsEstabelecimentos)

insertFiles(fileSimples, "Simples", conn, columnsSimples)

In [25]:
insertFiles(fileEstabelecimentos, "Estabelecimentos", conn, columnsEstabelecimentos)

Begin insert operation
Cursor avaliable
checking e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE
Inserting e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE
a
b
[CNPJ BÁSICO] BIGINT, [CNPJ ORDEM] BIGINT, [CNPJ DV] BIGINT, [IDENTIFICADOR MATRIZ/FILIAL] BIGINT, [NOME FANTASIA] NVARCHAR(600), [SITUAÇÃO CADASTRAL] BIGINT, [DATA SITUAÇÃO CADASTRAL] BIGINT, [MOTIVO SITUAÇÃO CADASTRAL] BIGINT, [NOME DA CIDADE NO EXTERIOR] FLOAT, [PAIS] FLOAT, [DATA DE INÍCIO ATIVIDADE] BIGINT, [CNAE FISCAL PRINCIPAL] BIGINT, [CNAE FISCAL SECUNDÁRIA] NVARCHAR(600), [TIPO DE LOGRADOURO] NVARCHAR(600), [LOGRADOURO] NVARCHAR(600), [NÚMERO] NVARCHAR(600), [COMPLEMENTO] NVARCHAR(600), [BAIRRO] NVARCHAR(600), [CEP] BIGINT, [UF] NVARCHAR(600), [MUNICÍPIO] BIGINT, [DDD 1] BIGINT, [TELEFONE 1] BIGINT, [DDD 2] FLOAT, [TELEFONE 2] FLOAT, [DDD DO FAX] FLOAT, [FAX] FLOAT, [CORREIO ELETRÔNICO] NVARCHAR

In [16]:
x = pd.read_csv("e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE", sep=';', names=columnsEstabelecimentos, nrows=19437, encoding='latin1')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Vitor\AppData\Local\Temp\ipykernel_12700\3611937997.py:1: SyntaxWarning: invalid escape sequence '\D'
  x = pd.read_csv("e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE", sep=';', names=columnsEstabelecimentos, nrows=19437, encoding='latin1')


In [17]:
x.tail()

,CNPJ BÁSICO,CNPJ ORDEM,CNPJ DV,IDENTIFICADOR MATRIZ/FILIAL,NOME FANTASIA,SITUAÇÃO CADASTRAL,DATA SITUAÇÃO CADASTRAL,MOTIVO SITUAÇÃO CADASTRAL,NOME DA CIDADE NO EXTERIOR,PAIS,...,MUNICÍPIO,DDD 1,TELEFONE 1,DDD 2,TELEFONE 2,DDD DO FAX,FAX,CORREIO ELETRÔNICO,SITUAÇÃO ESPECIAL,DATA DA SITUAÇÃO ESPECIAL
19432,20509697,1,38,1,NaN,8,20180201,67,NaN,NaN,...,5865,21.0,37047214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19433,8302453,1,84,1,PREGAO GOIAS,8,20170627,1,NaN,NaN,...,1050,62.0,35031619,62.0,39453291.0,62.0,39453291,jmonteirocontab@terra.com.br,NaN,NaN
19434,8302589,1,94,1,NaN,8,20150209,73,NaN,NaN,...,5803,22.0,000-0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19435,8302750,1,20,1,RODOMAIS TRANSPORTES,2,20060921,0,NaN,NaN,...,9051,67.0,99142212,NaN,NaN,NaN,NaN,RODOMAIS@RODOMAIS.COM.BR,NaN,NaN
19436,8302891,1,42,1,NaN,8,20110816,37,NaN,NaN,...,6193,14.0,35831242,14.0,35831188.0,14.0,35831342,rogeriogarzim@uol.com.br,NaN,NaN


In [18]:
print(type(x["TELEFONE 1"][19434]))

<class 'str'>


In [21]:
pd.api.types.is_float_dtype(x["TELEFONE 1"])

False

In [ ]:
with open("e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE", "r", encoding="latin1") as f:
    total_lines = sum(1 for _ in f)

In [32]:
df_sample = pd.read_csv(
    "e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE",
    sep=';',
    names=columnsEstabelecimentos,
    encoding='latin1',
    skiprows=range(1, total_lines-1000),  # pula todas menos cabeçalho + últimas 100
    nrows=1000
)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Vitor\AppData\Local\Temp\ipykernel_10404\2721388723.py:2: SyntaxWarning: invalid escape sequence '\D'
  "e:\Documentos\TCC\DetectordeViabilidade\TratamentoInicial\EstabelecimentosUnzip\K3241.K03200Y0.D50913.ESTABELE",


In [27]:
df_sample.head()

,CNPJ BÁSICO,CNPJ ORDEM,CNPJ DV,IDENTIFICADOR MATRIZ/FILIAL,NOME FANTASIA,SITUAÇÃO CADASTRAL,DATA SITUAÇÃO CADASTRAL,MOTIVO SITUAÇÃO CADASTRAL,NOME DA CIDADE NO EXTERIOR,PAIS,...,MUNICÍPIO,DDD 1,TELEFONE 1,DDD 2,TELEFONE 2,DDD DO FAX,FAX,CORREIO ELETRÔNICO,SITUAÇÃO ESPECIAL,DATA DA SITUAÇÃO ESPECIAL
0,15164610,1,6,1,LIBENTE MOVEIS PLANEJADOS,4,20231106,63,NaN,NaN,...,9221,62,39435544,62.0,92110630.0,62.0,30983336.0,contex@hotmail.com,NaN,NaN
1,62706963,1,62,1,NaN,2,20250913,0,NaN,NaN,...,7535,41,98361149,NaN,NaN,NaN,NaN,MMMSOUZA150323@GMAIL.COM,NaN,NaN
2,62706975,1,97,1,NaN,2,20250913,0,NaN,NaN,...,139,68,92142077,NaN,NaN,NaN,NaN,SARAHPANDA199406@GMAIL.COM,NaN,NaN
3,62706986,1,77,1,NaN,2,20250913,0,NaN,NaN,...,7057,11,98521834,NaN,NaN,NaN,NaN,OLELEZGAB@GMAIL.COM,NaN,NaN
4,62706996,1,2,1,NaN,2,20250913,0,NaN,NaN,...,6549,15,96105782,NaN,NaN,NaN,NaN,FLAVIA_COSTA_OLIVEIRA@HOTMAIL.COM,NaN,NaN


In [38]:
pd.api.types.is_float_dtype(df_sample["CNPJ BÁSICO"][0])

False